## 2.1 RSS를 이용하여 앱스토어 리뷰 크롤링 (XML 형식)

In [1]:
# 모듈 로딩

import pandas as pd #크롤링한 데이터를 데이터프레임으로 담아줄 것

import xmltodict #xml 파싱을 위한 모듈
from urllib.request import urlopen # url을 이용하여 웹상의 데이터를 가져올 irllib.request
import json #xmltodict의 결과물을 dict로 바꾸기 위함 

### 2.1.1 Get Data from URL (page 1 파싱)

In [2]:
# 하단 url로 접속해서 xml데이터를 파싱할 것임 
#reopen 함수를 사용하여 해당urk이 가지는 값을 가져온다
url = 'https://itunes.apple.com/kr/rss/customerreviews/page=1/id=884043462/sortby=mostrecent/xml?urlDesc=/customerreviews/id=884043462/sortBy=mostRecent/xml'
response = urlopen(url).read()
response

b'<?xml version="1.0" encoding="utf-8"?>\n\n\t<feed xmlns:im="http://itunes.apple.com/rss" xmlns="http://www.w3.org/2005/Atom" xml:lang="ko">\n\t\t<id>https://mzstoreservices-int.dslb.apple.com/kr/rss/customerreviews/page=1/id=884043462/sortby=mostrecent/xml?urlDesc=/customerreviews/id=884043462/sortBy=mostRecent/xml</id><title>iTunes Store:\xec\x82\xac\xec\x9a\xa9\xec\x9e\x90 \xed\x9b\x84\xea\xb8\xb0</title><updated>2022-08-03T03:42:48-07:00</updated><link rel="alternate" type="text/html" href="https://apps.apple.com/WebObjects/MZStore.woa/wa/viewGrouping?cc=kr&amp;id=29141"/><link rel="self" href="https://mzstoreservices-int.dslb.apple.com/kr/rss/customerreviews/page=1/id=884043462/sortby=mostrecent/xml?urlDesc=/customerreviews/id=884043462/sortBy=mostRecent/xml"/><link rel="first" href="https://itunes.apple.com/kr/rss/customerreviews/page=1/id=884043462/sortby=mostrecent/xml?urlDesc=/customerreviews/page=1/id=884043462/sortby=mostrecent/xml"/><link rel="last" href="https://itunes.ap

### 2.1.2 Parsing XML

In [3]:
xml = xmltodict.parse(response)
XmlToJson = json.loads(json.dumps(xml))

### 2.1.3 파싱한 xml 데이터를 내가 원하는대로 바꾸기

In [4]:
res_dict = []

for i in range(len(XmlToJson['feed']['entry'])):
    res_dict.append({
        'DATE' : XmlToJson['feed']['entry'][i]['updated'],
        'STAR' : XmlToJson['feed']['entry'][i]['im:rating'],
        'LIKE' : XmlToJson['feed']['entry'][i]['im:voteSum'],
        'DISLIKE' : int(XmlToJson['feed']['entry'][i]['im:voteCount']) - int(XmlToJson['feed']['entry'][i]['im:voteSum']),
        'TITLE' : XmlToJson['feed']['entry'][i]['title'],
        'REVIEW' : XmlToJson['feed']['entry'][i]['content'][0]['#text']
    })

In [5]:
# 잘 변환 되었는지 확인
res_dict

[{'DATE': '2022-08-01T23:14:22-07:00',
  'STAR': '5',
  'LIKE': '0',
  'DISLIKE': 0,
  'TITLE': '믿고 쓰는 여기어때 최고임니다',
  'REVIEW': '유용하게 잘 쓰고 잇슴니다\n필수 어플입니다 \n이벤트도 많고 짱!'},
 {'DATE': '2022-08-01T22:02:50-07:00',
  'STAR': '5',
  'LIKE': '0',
  'DISLIKE': 0,
  'TITLE': '원래는 경쟁사만 썼는데 이젠 여기어때만 씁니다!',
  'REVIEW': '일단 리뷰보는 게 진짜 편해요! 개인적으로 방 사진들 보다는 리뷰사진을 엄청 신뢰하는 편인데 리뷰 보는 게 편하고 결제까지 넘어가는 것도 너무 좋아가지고 저는 여기어때 정착합니당🤍'},
 {'DATE': '2022-08-01T18:07:33-07:00',
  'STAR': '5',
  'LIKE': '0',
  'DISLIKE': 0,
  'TITLE': '숙박예약은 여기어때로!',
  'REVIEW': '항상 여행계획을 짜게되면 여기어때부터 키고 봅니다\n숙소들을 한눈에 다 볼 수있어서 편리하고 한번도 꽝인 숙소가 없었던거 같아요ㅎㅎ이번달도 가족들과 여름휴가로 어디갈지 숙소부터 찾아보고있어요~'},
 {'DATE': '2022-08-01T17:59:38-07:00',
  'STAR': '1',
  'LIKE': '0',
  'DISLIKE': 0,
  'TITLE': '여기어때가 여기어때 하네요',
  'REVIEW': '고객센터 상담원 연결 시도 3일이 걸려 예약자명 변경이 안돼 영수증 발급 까지 지금 몇일이 걸리는지.. 기본적인 업무도 고객센터에서 확인해주지 않구 "불가능합니다 안됩니다" 이런 답변만.. \n 설명 부족으로 하루 30만원 하는 방을 2개를 잡아서 1개를 빈방으로 썼네요 여기어때는 앞으로 이용 안할 예정이구요 꿀스테이 좋다고해서 갈아 타볼 예정입니다'},
 {'DATE': '2022-08-01T17

In [6]:
#ㅔpage 1 파싱 내용을 데이터프레임으로 담아줍니다 
res_df = pd.DataFrame(res_dict)
res_df

,DATE,STAR,LIKE,DISLIKE,TITLE,REVIEW
0,2022-08-01T23:14:22-07:00,5,0,0,믿고 쓰는 여기어때 최고임니다,유용하게 잘 쓰고 잇슴니다\n필수 어플입니다 \n이벤트도 많고 짱!
1,2022-08-01T22:02:50-07:00,5,0,0,원래는 경쟁사만 썼는데 이젠 여기어때만 씁니다!,일단 리뷰보는 게 진짜 편해요! 개인적으로 방 사진들 보다는 리뷰사진을 엄청 신뢰하...
2,2022-08-01T18:07:33-07:00,5,0,0,숙박예약은 여기어때로!,항상 여행계획을 짜게되면 여기어때부터 키고 봅니다\n숙소들을 한눈에 다 볼 수있어서...
3,2022-08-01T17:59:38-07:00,1,0,0,여기어때가 여기어때 하네요,고객센터 상담원 연결 시도 3일이 걸려 예약자명 변경이 안돼 영수증 발급 까지 지금...
4,2022-08-01T17:16:09-07:00,5,0,0,어디 놀러갈때마다 이용중,너무 좋아요 예약하기도 싶고 할인이벤도 마니하고
5,2022-08-01T07:38:40-07:00,1,0,0,진작에 리뷰 볼걸,고객센터 엉망이라고 리뷰가 많았네요 안내받은 방의 상태가 설명과 달라 환불요청드렸더...
6,2022-08-01T01:40:05-07:00,1,0,0,등신드라 예약가능한 숙소부터 보여줘야지,날짜 설정했는데 예약 마감인거 부터 보여주냐 무뇌새기들
7,2022-08-01T00:50:49-07:00,5,0,0,수야,엄청 좋아요 !
8,2022-07-31T22:13:43-07:00,5,0,0,야널자에서 갈아탐,야널자엔 매일 무료 룰렛도 없고 ㅋ 심지어 여기어때에서ㅠ풀려있는 숙소가 막혀있음
9,2022-07-31T17:32:37-07:00,1,0,0,소비자 보호 없는 외국계기업,소비자가 피해를 입어도 해결 의지 절대 없는 여기어때 소비자보호원에서도 여기어때는 ...


In [7]:
#시계열 컬럼을 예쁘게 수정해주기
res_df['DATE'] = pd.to_datetime(res_df['DATE'], format="%Y-%m-%dT%H:%M:%S-07:00")
res_df.head()

,DATE,STAR,LIKE,DISLIKE,TITLE,REVIEW
0,2022-08-01 23:14:22,5,0,0,믿고 쓰는 여기어때 최고임니다,유용하게 잘 쓰고 잇슴니다\n필수 어플입니다 \n이벤트도 많고 짱!
1,2022-08-01 22:02:50,5,0,0,원래는 경쟁사만 썼는데 이젠 여기어때만 씁니다!,일단 리뷰보는 게 진짜 편해요! 개인적으로 방 사진들 보다는 리뷰사진을 엄청 신뢰하...
2,2022-08-01 18:07:33,5,0,0,숙박예약은 여기어때로!,항상 여행계획을 짜게되면 여기어때부터 키고 봅니다\n숙소들을 한눈에 다 볼 수있어서...
3,2022-08-01 17:59:38,1,0,0,여기어때가 여기어때 하네요,고객센터 상담원 연결 시도 3일이 걸려 예약자명 변경이 안돼 영수증 발급 까지 지금...
4,2022-08-01 17:16:09,5,0,0,어디 놀러갈때마다 이용중,너무 좋아요 예약하기도 싶고 할인이벤도 마니하고


### 2.1.4 반복문으로 page 1~ 10 까지 한번에 긁어오기

In [11]:
res_dict = []
for j in range(1,11):
    url = 'https://itunes.apple.com/kr/rss/customerreviews/page=%i/id=884043462/sortby=mostrecent/xml?urlDesc=/customerreviews/id=884043462/sortBy=mostRecent/xml' % j
    response = urlopen(url).read()

    xml = xmltodict.parse(response)
    XmlToJson = json.loads(json.dumps(xml))

    for i in range(len(XmlToJson['feed']['entry'])):
        res_dict.append({
            'DATE' : XmlToJson['feed']['entry'][i]['updated'],
            'STAR' : int(XmlToJson['feed']['entry'][i]['im:rating']),
            'LIKE' : int(XmlToJson['feed']['entry'][i]['im:voteSum']),
            'DISLIKE' : int(XmlToJson['feed']['entry'][i]['im:voteCount']) - int(XmlToJson['feed']['entry'][i]['im:voteSum']),
            'TITLE' : XmlToJson['feed']['entry'][i]['title'],
            'REVIEW' : XmlToJson['feed']['entry'][i]['content'][0]['#text']
        })

res_df = pd.DataFrame(res_dict)
res_df['DATE'] = pd.to_datetime(res_df['DATE'], format="%Y-%m-%dT%H:%M:%S-07:00")
#csv로 저장
res_df.to_csv('data/여기어때_review.csv', index = False, encoding = 'utf-8')

In [12]:
#잘 저장 되었는지 확인하기
df = pd.read_csv('data/여기어때_review.csv', encoding = 'utf-8')
df.head()

,DATE,STAR,LIKE,DISLIKE,TITLE,REVIEW
0,2022-08-01 23:14:22,5,0,0,믿고 쓰는 여기어때 최고임니다,유용하게 잘 쓰고 잇슴니다\n필수 어플입니다 \n이벤트도 많고 짱!
1,2022-08-01 22:02:50,5,0,0,원래는 경쟁사만 썼는데 이젠 여기어때만 씁니다!,일단 리뷰보는 게 진짜 편해요! 개인적으로 방 사진들 보다는 리뷰사진을 엄청 신뢰하...
2,2022-08-01 18:07:33,5,0,0,숙박예약은 여기어때로!,항상 여행계획을 짜게되면 여기어때부터 키고 봅니다\n숙소들을 한눈에 다 볼 수있어서...
3,2022-08-01 17:59:38,1,0,0,여기어때가 여기어때 하네요,고객센터 상담원 연결 시도 3일이 걸려 예약자명 변경이 안돼 영수증 발급 까지 지금...
4,2022-08-01 17:16:09,5,0,0,어디 놀러갈때마다 이용중,너무 좋아요 예약하기도 싶고 할인이벤도 마니하고


In [13]:
df['DATE'].sort_values(ascending = True)

499    2022-01-06 03:15:54
498    2022-01-06 03:29:58
497    2022-01-06 06:20:39
496    2022-01-06 15:56:09
495    2022-01-06 22:24:50
              ...         
4      2022-08-01 17:16:09
3      2022-08-01 17:59:38
2      2022-08-01 18:07:33
1      2022-08-01 22:02:50
0      2022-08-01 23:14:22
Name: DATE, Length: 500, dtype: object